

attributes (class versus instance), method resolution order
getattr setattr
properties and descriptors


link it to behavior and state

setters are methods for updating state, while methods in python should be used for behavior

so can we update state as an assignment (rather than method invocation)?

other use cases for this


Agents are objects, so they are characterized by state and behavior. State is handled through attributes, while behavior is handled through methods. But what about the following situation: we would like to update the state of an object, but the possible values that this state might take is constrained. We would ideally like to raise an error if the state is being updated to an incorrect number. How can we do this in Python?

To make this question more concrete, consider the following example. We have a Temperature class which stores temperature in Celsius, and from there can convert it into both Kelvin and Fahrenheit. The lowest possible temperature in Celcius is -273.15. So if we were to pass a value lower then this, we would want to see an error. Let's start with a simple implementation and take it from there.

In [5]:
class Celsius:
    
    def __init__(self, temperature=0):
        self.temperature = temperature
        
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    
    def to_kelvin(self):
        return self.temperature + 273.15
    
temperature = Celsius()
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

32.0
273.15


So far, so good. But what happens if we try to create a Celcius instance with -300?

In [8]:
temperature = Celsius(temperature=-300)
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

-508.0
-26.850000000000023


We get negative Kelvin, which of course can't happen. So how to fix this? An easy first fix would be to check explicitly whether the temperature keyword argument in the `__init__` is valid and raise an error otherwise. 

In [12]:
class Celsius:
    lowest = -273.15
    
    def __init__(self, temperature=0):
        if temperature < self.lowest:
            raise ValueError("temperature cannot be lower "
                             f"then {self.lowest} Celsius")
        
        self.temperature = temperature
        
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    
    def to_kelvin(self):
        return self.temperature + (-1*self.lowest)
    
temperature = Celsius()
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

    
temperature = Celsius(temperature=-300)
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

32.0
273.15


ValueError: temperature cannot be lower then -273.15 Celsius

This simple check seem to have fixed the problem. However, what happens if we try to update the value of the temperature attribute after having created the Celsius instance?

In [13]:
temperature = Celsius()
temperature.temperature = -300
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

-508.0
-26.850000000000023


This does not raise an error at all. The explanation for this is that we only check if the temperature value is valid when we instantiate the Celsius object, but not when updating the value of the attribute. Ideally we would want to check in both situations. How can we do this?

A first simple idea would be to have a method that we can use for updating the temperature like this

In [17]:
class Celsius:
    lowest = -273.15
    
    def __init__(self, temperature=0):
        self.set_temperature(temperature)
    
    def set_temperature(self, temperature):
        if temperature < self.lowest:
            raise ValueError("temperature cannot be lower "
                             f"then {self.lowest} Celsius")
        
        self.temperature = temperature        
    
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    
    def to_kelvin(self):
        return self.temperature + (-1*self.lowest)
    
temperature = Celsius()
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

    
temperature = Celsius(temperature=-300)
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

32.0
273.15


ValueError: temperature cannot be lower then -273.15 Celsius

In [19]:
temperature = Celsius()
temperature.set_temperature(-300)
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

ValueError: temperature cannot be lower then -273.15 Celsius

So adding the method for setting temperature seems to work fine. Note however 3 things;
1. your user now has to rememeber to always use the `set_temperature` method instead of the much simpler attribute assignment
2. you are using a method, which is intended for representing behavior, to update state.
3. you can still easily create an error if you forget to use the method as shown below.

In [20]:
temperature = Celsius()
temperature.temperature = -300
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

-508.0
-26.850000000000023


So our fix did not really work. Ideally, we want to be able to use attribute assignment (i.e., `temperature.temperature. = -250`), yet still have this trigger a small bit of evaluative code for the check. Can this be done in Python?

Yes we can do this in Python. There is a special language feature known as **properties** that allows us to do exactly this. The simplest way to use properties in Python is by using the `@property` annotation. Annotations are another Python language feature. Annotations are in essence convenient programming shorthands, or syntactic sugar, to make implementing something requiring less code. Fully understanding annotations, how they work, and how to create your own is well beyond this course. Outside of the `@property` annotations, you will not encounter them in this course. 

If we want to say that a given attribute is a property, we need to specify a setter method and a getter method. The getter uses the `@property` annotation. The name of the method underneath the annotation becomes the name of the property. Next, we also have to create a setter method and link this to our property. We do this by using the `@n{name of property}.setter` syntax. Whenever we try to retrieve `temperature` as if it is an attribute, Python will actually call the `temperature` method for us. Likewise, whenever we want to assign a new value to the attribute, Python will actually call the setter method for us. 

A last thing to note is that we cannot have an attribute and a property with the same name on a class. Since we want to use temperature as the name of the property, we need to assign the value to an attribute that is named differently. It is convention in Python to use `_{name of property}` for this, so `_temperature` in this case. 

In [26]:
class Celsius:
    lowest = -273.15
    
    @property
    def temperature(self):
        return self._temperature
    
    @temperature.setter
    def temperature(self, temperature):
        if temperature < self.lowest:
            raise ValueError("temperature cannot be lower "
                             f"then {self.lowest} Celsius")

        self._temperature = temperature            
    
    
    def __init__(self, temperature=0):
        self.temperature = temperature
    
    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32
    
    def to_kelvin(self):
        return self.temperature + (-1*self.lowest)
    
temperature = Celsius()
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

    
temperature = Celsius(temperature=-300)
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

32.0
273.15


ValueError: temperature cannot be lower then -273.15 Celsius

In [27]:
temperature = Celsius()
temperature.temperature = -300
print(temperature.to_fahrenheit())
print(temperature.to_kelvin())

ValueError: temperature cannot be lower then -273.15 Celsius

Let us recap. we started this tutorial with the following problem: can we update the state of an object but while updating this check if the provided new value is valid? As we have seen, yes this is possible in Python by using properties. 

However, checking the validity of values when changing state is but one of the use cases for properties. Two other relevant use cases are
1. various attributes are directly related to each other. For example, you have an agent moving on a circle. You can express its position in radians but for e.g., visualization you want to know the x and y coordinates. You can then use properties for the x and y coordinates that on the fly calculate them when needed.
2. you initially implemented something as an attribute but later realize you need to add some code to it as well. Changing the attribute to a method would require changes elsewhere in your code as well. Properties allow you to make the change in one place only and not braking the rest of your code base
